# How to run TARDIS with a custom ejecta model

This notebook will go through multiple detailed examples of how to properly run TARDIS with a custom ejecta profile specified by a custom density file and a custom abundance file. 

In [1]:
import tardis
import matplotlib.pyplot as plt
import numpy as np

## Your custom density file

First, let's look at an example of a custom density file.

* The first line specifies the time in days after the explosion
* After a skipped line, each row corresponds to a shell with index specified by the first column.
* The second column lists the velocities of the outer boundary of the cell in km/s.
* The third column lists the density of the cell.

### <font color=red>Important: </font>
The __default behavior__ of TARDIS is to use the first shell as the inner boundary. This means that v_inner_boundary = 9500, and the corresponding density 9e-16 is ignored because it is within the inner boundary. It can be replaced by an arbitrary number.  The outer boundary of the last shell will be used as v_outer_boundary, so the default behavior will set v_outer_boundary = 12000.

## Your custom abundance file

Let's look at an example of a custom density file.

* The first line indicates which elements (or isotopes) correspond to which columns.
* After a skipped line, each row specifies the chemical abundance of one shell. Therefore the numbers in a given row should sum to 1.0

### <font color=red>Important: </font>
Note that there are only 2 shells specified in this abundance file (despite the custom density file having 3 lines). This is because the custom density file specifies the boundaries of the shells, while the abundance file specifies the abundances within each shell.

## Running TARDIS with the custom files

Now let's run TARDIS using the example custom files.

In [2]:
model = tardis.run_tardis('./test_config.yml')

[py.warnings         ][WARNING]  /home/mew488/anaconda2/envs/tardis3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
 (warnings.py:99)
[py.warnings         ][WARNING]  /home/mew488/anaconda2/envs/tardis3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
 (warnings.py:99)
[tardis.model.base   ][WARNING]  Abundances have not been normalized to 1. - normalizing (base.py:367)
[py.warnings         ][WARNING]  /home/mew488/anaconda2/envs/tardis3/lib/python3.6/site-packages/astropy/units/quantity.py:1067: AstropyDeprecationWarning: The truth value of a Quantity is ambiguous. In the future this will raise a ValueError.
  AstropyDeprecationWarning)
 (warnings.py:99)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from /home/mew488/Research/TARDIS/tardis-refdata/atom_data/kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:77)
[py.warnings         ][

[tardis.simulation.base][INFO   ]  t_inner 6551.873 K -- next t_inner 6482.711 K (base.py:350)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:266)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 2.21723e+42 erg / s Luminosity absorbed = 2.12445e+38 erg / s Luminosity requested = 2.26469e+42 erg / s (base.py:357)
[tardis.simulation.base][INFO   ]  Iteration converged 8/4 consecutive times. (base.py:194)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad  next_t_rad         w    next_w
	Shell                                             
	0      6586.535643   6452.0629  0.344429  0.358012

 (base.py:348)
[tardis.simulation.base][INFO   ]  t_inner 6482.711 K -- next t_inner 6551.731 K (base.py:350)
[tardis.simulation.base][INFO   ]  Starting iteration 9/20 (base.py:266)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 2.31306e+42 erg / s Luminosity absorbed = 3.30756e+38 erg / s Luminosity requested = 2.26469e+42 erg / s (base.

[tardis.simulation.base][INFO   ]  Luminosity emitted = 2.21806e+42 erg / s Luminosity absorbed = 3.18123e+38 erg / s Luminosity requested = 2.26469e+42 erg / s (base.py:357)
[tardis.simulation.base][INFO   ]  Iteration converged 18/4 consecutive times. (base.py:194)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	            t_rad   next_t_rad         w    next_w
	Shell                                             
	0      6541.53332  6506.188917  0.354308  0.346155

 (base.py:348)
[tardis.simulation.base][INFO   ]  t_inner 6483.406 K -- next t_inner 6551.206 K (base.py:350)
[tardis.simulation.base][INFO   ]  Starting iteration 19/20 (base.py:266)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 2.31238e+42 erg / s Luminosity absorbed = 2.76543e+38 erg / s Luminosity requested = 2.26469e+42 erg / s (base.py:357)
[tardis.simulation.base][INFO   ]  Iteration converged 19/4 consecutive times. (base.py:194)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	      

You can check to make sure that the model loaded and used by TARDIS during the simulation is consistent with your expectations based on the custom files you provided:

In [3]:
print('v_inner_boundary = ',model.model.v_boundary_inner)
print('v_outer_boundary = ',model.model.v_boundary_outer)
print('\n')
print('velocities of shell boundaries: ')
print(model.model.velocity)
print('\n')
print('densities loaded by TARDIS: (NOTE that the density in the first line of the file was ignored! Densities are also rescaled.)')
print(model.model.density)

v_inner_boundary =  950000000.0 cm / s
v_outer_boundary =  1200000000.0 cm / s


velocities of shell boundaries: 
[9.50e+08 1.05e+09 1.20e+09] cm / s


densities loaded by TARDIS: (NOTE that the density in the first line of the file was ignored! Densities are also rescaled.)
[7.5e-14 2.5e-15] g / cm3


## Specifying boundary velocities in the config file

In addition to specifying custom density and abundance files, the user can set the v_inner_boundary and v_outer_boundary velocities in the YAML config file. This can cause some confusion, so we carefully go through some examples.

### <font color=red>Important: </font>
Boundary velocities set in the YAML config file must be __within__ the velocity range specified in the custom density file (if one is provided).

## Example 1) v_inner_boundary lower than first velocity in density file

In this example, the first velocity in the density file is 9500 km/s. The user can specify in the config file the velocity of the inner boundary to a lower velocity, say v_inner_boundary = 9000 km/s. This will cause TARDIS to raise an error.

In [4]:
model = tardis.run_tardis('./test_config_ex1.yml')

[tardis.model.base   ][WARNING]  Abundances have not been normalized to 1. - normalizing (base.py:367)


ValueError: v_boundary_inner is lower than the lowest shell in the model.

## Example 2) v_outer_boundary larger than last velocity in density file

In this example, the last velocity in the density file is 12000 km/s. The user can specify in the config file the velocity of the outer boundary to a larger velocity, say v_outer_boundary = 13000 km/s. This will cause TARDIS to raise an error.

In [5]:
model = tardis.run_tardis('./test_config_ex2.yml')

[tardis.model.base   ][WARNING]  Abundances have not been normalized to 1. - normalizing (base.py:367)


ValueError: v_boundary_outer is larger than the largest shell in the model.

## Example 3) v_boundaries in config file are within density file velocity range

Here the user sets v_inner_boundary = 9700 and v_outer_boundary = 11500 in the config file. Both values fall within the velocity range specified by the custom density file.

In [6]:
model = tardis.run_tardis('./test_config_ex3.yml')

[tardis.model.base   ][WARNING]  Abundances have not been normalized to 1. - normalizing (base.py:367)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from /home/mew488/Research/TARDIS/tardis-refdata/atom_data/kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:77)
[py.warnings         ][WARNING]  /home/mew488/anaconda2/envs/tardis3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3291: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
 (warnings.py:99)
[tardis.io.atomic    ][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (atomic.py:173)
[tardis.io.atomic    ][INFO   ]  Non provided atomic data: synpp_refs, ion_cx_th_data, ion_cx_sp_data (atomic.py:176)
[py.warnings         ][WARNING]  /home/mew488/src/dev/tardis/tardis/plasma/properties/ion_population.py:59: FutureWarning: 
Passing list-likes to .loc or [] with any missing lab

[tardis.simulation.base][INFO   ]  t_inner 6414.228 K -- next t_inner 6483.748 K (base.py:350)
[tardis.simulation.base][INFO   ]  Starting iteration 9/20 (base.py:266)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 2.31428e+42 erg / s Luminosity absorbed = 5.54016e+37 erg / s Luminosity requested = 2.26469e+42 erg / s (base.py:357)
[tardis.simulation.base][INFO   ]  Iteration converged 9/4 consecutive times. (base.py:194)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad   next_t_rad         w    next_w
	Shell                                              
	0      6383.889203  6473.974678  0.374686  0.369973

 (base.py:348)
[tardis.simulation.base][INFO   ]  t_inner 6483.748 K -- next t_inner 6413.915 K (base.py:350)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:266)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 2.21596e+42 erg / s Luminosity absorbed = 2.63810e+38 erg / s Luminosity requested = 2.26469e+42 erg / s (b

[tardis.simulation.base][INFO   ]  Luminosity emitted = 2.31299e+42 erg / s Luminosity absorbed = 2.20411e+38 erg / s Luminosity requested = 2.26469e+42 erg / s (base.py:357)
[tardis.simulation.base][INFO   ]  Iteration converged 19/4 consecutive times. (base.py:194)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad   next_t_rad         w    next_w
	Shell                                              
	0      6436.504989  6520.140466  0.362441  0.358913

 (base.py:348)
[tardis.simulation.base][INFO   ]  t_inner 6482.959 K -- next t_inner 6414.912 K (base.py:350)
[tardis.simulation.base][INFO   ]  Starting iteration 20/20 (base.py:266)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 2.21744e+42 erg / s Luminosity absorbed = 1.90600e+38 erg / s Luminosity requested = 2.26469e+42 erg / s (base.py:357)
[tardis.simulation.base][INFO   ]  Simulation finished in 20 iterations and took 2.44 s (base.py:306)


In [7]:
print('v_inner_boundary = ',model.model.v_boundary_inner)
print('v_outer_boundary = ',model.model.v_boundary_outer)
print('\n')
print('velocities of shell boundaries: ')
print(model.model.velocity)
print('\n')
print('densities loaded by TARDIS: (NOTE that the density in the first line of the file was ignored! Densities are also rescaled.)')
print(model.model.density)

v_inner_boundary =  970000000.0 cm / s
v_outer_boundary =  1150000000.0 cm / s


velocities of shell boundaries: 
[9.70e+08 1.05e+09 1.15e+09] cm / s


densities loaded by TARDIS: (NOTE that the density in the first line of the file was ignored! Densities are also rescaled.)
[7.5e-14 2.5e-15] g / cm3


### <font color=red>Important: </font>
Notice that the inner and outer boundary velocities are the ones specifically set by the user.